In [ ]:
pip install openpyxl

In [ ]:
import glob 
import pandas as pd
import os
import openpyxl
import requests
from bs4 import BeautifulSoup as BS 
from tqdm import tqdm

In [ ]:
ggg = pd.read_csv('./miniproject/2201.csv', encoding='cp949')

In [ ]:
target_csv = sorted(glob.glob("./miniproject/*.csv"), key=os.path.getctime)

In [ ]:
csv_test = pd.read_csv(target_csv[0],encoding='cp949')

In [ ]:
import pickle
dfs = []
for fff in target_csv:
    df = pd.read_csv(fff,encoding='cp949')
    dfs.append(df)

dfs[6]['대여일자'] = dfs[6]['대여일자'].apply(lambda x : str(x)[:4] + "-" + str(x)[4:])
ddd = pd.concat(dfs, ignore_index=True)
num_2022 = ddd.groupby(['대여일자', '대여소번호'], as_index=False)['이용건수'].sum()
# dfs[6] 의 숫자들을 2022-07 형식으로만들기
# 콘캣후에 이용건수 합치기 - 월별
# 대여일자,대여소번호,이용건수 나열
# 대여일자만 내용이달라 하이픈추가하였음

In [ ]:
with open("./CycleRentUseMonth_2022.pkl", "wb") as f:
    pickle.dump(num_2022, f)

In [ ]:
sql6 ="INSERT  INTO CycleRentUse VALUES (%s, %s, %s)"

In [ ]:
num_2022.apply(lambda x : cur.execute(sql6, x.tolist() ), axis=1)

In [ ]:
con.commit()

In [ ]:
url = "https://tmacs.kotsa.or.kr/web/TG/TG200/TG2100S/Tgsearch01_AJAX.jsp"
payload = {"gubun": "S1181",
"mid": "S1181",
"sYear": "2022",
"eYear": "2022",
"sido": "11000",
"jijace": "11680",
"kind1": "s102",
"kind2": "s1021",
"cartype": "07"}

year_list=["2018","2019","2020","2021","2022"]
jijace_list=[["강남구" , 11680] , ["강동구" ,11740], ["강북구" , 11305], ["강서구" , 11500],  ["관악구", 11620], ["광진구" , 11215], ["구로구" ,11530],
            ["금천구",11545], ["노원구" , 11350] ,["도봉구",11320], ["동대문구",11230], ["동작구" , 11590], ["마포구" ,11440] , ["서대문구" ,11410] , ["서초구", 11650],
            ["성동구" , 11200], ["성북구" , 11290], ["송파구" , 11710], ["양천구" ,11470], ["영등포구" , 11560], ["용산구" , 11170], ["은평구" , 11380],
            ["종로구", 11110] , ["중구" , 11140], ["중랑구",11260]]
jajun_df=pd.DataFrame()
for i in tqdm(range(len(jijace_list))):
    for j in range(len(year_list)):
        payload["sYear"]=year_list[j]
        payload["eYear"]=year_list[j]
        payload["jijace"]=jijace_list[i][1]
            # print(year_list[j])
            # print(jijace_list[i][1])
        r = requests.post(url, data=payload)
        bs = eval(r.text)
        df = pd.DataFrame(bs)
        df["YEAR"]=year_list[j]
        jajun_df = pd.concat([jajun_df, df], axis = 0)
            # pd.set_option('display.max_columns', None)


In [ ]:
#자전거만 따오는 작업

In [ ]:
 jajun_df.rename(columns = {'GUBUN_NM' : 'MONTH'}, inplace = True)

In [ ]:
jajun_df["BASE_CATEGORY"] = "운전경력"

In [ ]:
with open("./carexp_accident_cycle_month.pkl", "wb") as f:
    pickle.dump(jajun_df, f)

In [ ]:
sql7 = """CREATE Table carexp_accident_cycle_month
(
    JIJACE_NM VARCHAR(30),
    CHISA FLOAT,
    ACC_HUM INT,
    SIDO_CD INT,
    MONTH VARCHAR(2),
    SLTWD_CNT INT,
    SERI_ACC INT,
    SIDO_NM VARCHAR(10),
    AC_MODE VARCHAR(4),
    CRT_JIJACE_CD INT ,
    BASE_NAME VARCHAR(30),
    BIG_CNT INT,
    PSGT_CNT INT,
    WND_ACC INT,
    ACC_CNT INT,
    TRK_CNT INT,
    DEATH_CNT INT,
    BASE_CODE VARCHAR(10),
    WND_CNT INT,
    DEATH_ACC INT,
    GUBUN VARCHAR(5),
    SERI_CNT INT,
    SLTWD_ACC INT,
    YEAR INT,
    BASE_CATEGORY VARCHAR(10)
)"""

In [ ]:
cur.execute(sql7)
con.commit()

In [ ]:
sql7 ="INSERT  INTO carexp_accident_cycle_month VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s ,%s ,%s ,%s)"

In [ ]:
jajun_df.apply(lambda x : cur.execute(sql7, x.tolist() ), axis=1)

In [ ]:
con.commit()

In [ ]:
pip install seaborn

In [ ]:
jajun_df = jajun_df.astype({
    'MONTH':'int64'
})

In [ ]:
jajun_df['MONTH'].dtype

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
plt.figure(figsize=(30,6))
sns.barplot(x=jajun_df['MONTH'], y=jajun_df['ACC_CNT'], color='steelblue')
plt.title('Number of Bicycle Accident by Month')
plt.xlabel('MONTH')
plt.ylabel('Number of Accident')
plt.show()